# Final Project Proposal - Draft 1 - Vision Statement

### by Joseph Jinn

In [8]:
"""
Course: CS 344 - Artificial Intelligence
Instructor: Professor VanderLinden
Name: Joseph Jinn
Date: 4-23-19

Final Project - SLO Topic Classification

Notes:

TODO - placeholder

Resources Used:

TODO - placeholder

"""

################################################################################################################

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

################################################################################################################

# Import the dataset.
slo_dataset = \
    pd.read_csv("datasets/tbl_training_set.csv", sep=",")

# Shuffle the data randomly.
slo_dataset = slo_dataset.reindex(
    np.random.permutation(slo_dataset.index))

# Rename columns to something that makes sense.
column_names = ['Tweet', 'SLO1', 'SLO2', 'SLO3']
slo_dataset.columns = column_names

# Print shape and column names.
print("The shape of our SLO dataset:")
print(slo_dataset.shape)
print()
print("The columns of our SLO dataset:")
print(slo_dataset.head)
print()

# Create input features.
selected_features = slo_dataset[column_names]
processed_features = selected_features.copy()

print("The tweets as a string:")
print(processed_features['Tweet'])
print()
print("SLO classification tag 1 (if any):")
print(processed_features['SLO1'])
print()
print("SLO classification tag 2 (if any):")
print(processed_features['SLO2'])
print()
print("SLO classification tag 3 (if any):")
print(processed_features['SLO3'])
print()

############################################################################################

"""
Main function.  Execute the program.
"""
# Debug variable.
debug = 0

if __name__ == '__main__':
    print()

############################################################################################



The shape of our SLO dataset:
(299, 4)

The columns of our SLO dataset:
<bound method NDFrame.head of                                                  Tweet           SLO1    SLO2  \
144  RT @TFBOYS_THFANS: HQ 160528 Wangyuan @ CCTV C...            NaN     NaN   
229                    Cutie ! https://t.co/wBhpKvFvXy            NaN     NaN   
219  BHP-Vale joint venture ignored dam-collapse ri...         social     NaN   
95   RT @PaulSinclair01: .@AusConservation advert i...  environmental     NaN   
78   RT @FreedZones: LNP Adani rorting in their DNA...         social     NaN   
..                                                 ...            ...     ...   
153  RT @ASENenviro: Protestors march on #BHP Olymp...         social     NaN   
61   #Adani's #Carmichael mine will destroy our lan...  environmental  social   
283  RT @AustralisTerry: Looking forward to @RealBo...         social     NaN   
17   RT @arianewilkinson: Corporate background chec...  environmental  social   
188    